# Agent & Tools

## 도구 호출이란?
도구 호출(Tool Calling)은 LangChain에서 언어 모델이 외부 도구나 API를 사용할 수 있게 해주는 중요한 기능입니다. 
이를 통해 AI 모델은 실시간 정보 검색, 계산, 데이터베이스 쿼리 등 다양한 작업을 수행할 수 있습니다. 
특히, 에이전트(Agent) 기반의 시스템을 만들 때 매우 중요한 역할을 하기 때문에 잘 알아둘 필요가 있습니다.

## 도구 호출의 중요성
1. 기능 확장
   1. 언어 모델의 능력을 크게 확장시킵니다. 모델은 자체적으로 가지고 있지 않은 정보나 기능에 접근할 수 있습니다.
2. 실시간 정보 접근
   1. 웹 검색 도구를 통해 최신 정보를 얻을 수 있어, 항상 최신의 답변을 제공할 수 있습니다.
3. 정확성 향상
   1. 계산기 도구나 데이터베이스 쿼리 도구를 사용하여 정확한 수치나 데이터를 얻을 수 있습니다.
4. 다양한 작업 수행
   1. 코드 실행, 파일 조작, API 호출 등 다양한 작업을 수행할 수 있습니다.

## LangChain v1.0의 Agent 시스템
### 새로운 특징
1. 통합된 create_agent() API
   1. 이전 버전 
      1. initialize_agent() 함수와 복잡한 AgentType enum 사용
   2. 현재 버전 
      1. create_agent() 통합 API로 단순화 - AgentType enum 제거 - 더 직관적인 파라미터 구조
하나의 함수로 모든 타입의 에이전트를 생성할 수 있습니다.

2. 미들웨어 시스템
에이전트의 동작을 세밀하게 제어할 수 있는 미들웨어 시스템이 추가되었습니다.
- dynamic_prompt: 런타임에 동적으로 시스템 프롬프트 생성
- wrap_tool_call: 도구 호출을 래핑하여 에러 처리 및 로깅
- wrap_model_call: 모델 호출을 인터셉트하여 커스텀 로직 추가

3. 간결한 프롬프트 설정
system_prompt 파라미터로 프롬프트를 직접 설정할 수 있습니다.

4. TypedDict 기반 상태 스키마
타입 안정성이 강화되어 더 안전한 코드 작성이 가능합니다.

- 도구 호출의 작동 방식

In [ ]:
# 1. 도구 정의
# 개발자가 사용할 도구들을 정의하고 설정합니다.

from langchain.tools import tool

@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    # 웹 검색 로직
    return "검색 결과..."


In [ ]:
# 2. 에이전트 생성
# 정의한 도구들을 에이전트에 등록합니다.
agent = create_agent(
    model="gpt-4o",
    tools=[search_web],
    system_prompt="You are a helpful assistant."
)

3. 모델 추론
언어 모델이 주어진 태스크를 분석하고 필요한 도구를 결정합니다.

4. 도구 호출
모델이 선택한 도구를 호출하고 결과를 받아옵니다.

5. 결과 통합
도구 호출의 결과를 모델의 응답에 통합하여 최종 답변을 생성합니다.

In [3]:
# 기본 예제
# 날씨 조회 에이전트
from langchain.agents import create_agent
from langchain.tools import tool

# 1. 도구 정의
@tool
def get_weather(location: str) -> str:
    """Get weather information for a specific location.

    Args:
        location: The city or location name (e.g., "Seoul", "New York")

    Returns:
        Weather information as a string
    """
    print(location)
    # 실제로는 날씨 API를 호출하지만, 여기서는 예시로 고정값 반환
    weather_data = {
        "Seoul": "맑음, 25°C, 습도 60%",
        "New York": "흐림, 18°C, 습도 75%",
        "Tokyo": "비, 20°C, 습도 85%"
    }
    return weather_data

# 2. 에이전트 생성
agent = create_agent(
    model="gpt-4o",
    tools=[get_weather],
    system_prompt="""You are a helpful weather assistant.
When users ask about weather, use the get_weather tool to fetch current information.
Always provide the weather information in a friendly and clear manner."""
)

# 3. 에이전트 실행
result = agent.invoke({
    "messages": [{"role": "user", "content": "서울 날씨 어때?"}]
})

print(result["messages"][-1].content)


서울
서울의 현재 날씨는 맑고 기온은 25°C이며, 습도는 60%입니다. 좋은 하루 보내세요! ☀️


<details>
<summary>tool 함수에서 어떻게 location에 서울이 들어갔는가?</summary>

> `location`을 `print(location)` 했을 때 “서울”이 찍힌 이유는, **에이전트(LLM)가 사용자의 자연어 질문(“서울 날씨 어때?”)에서 핵심 인자(location)를 추출해서 `get_weather` 도구를 호출할 때 그 인자로 넘겼기 때문**입니다. 즉, “서울”은 코드 어딘가에서 하드코딩되어 자동으로 들어간 게 아니라, **모델이 대화 문맥을 이해해 “이 질문의 location은 서울이네”라고 판단해서** 도구 호출을 구성한 결과예요.
> 
> 다만, 질문에 올려주신 예제 코드는 엄밀히 말하면 몇 가지가 섞여 있습니다:
> 
> - `langchain.agents.create_agent`는 LangChain 공식 API와 이름이 같을 수 있지만, 실제로는 **사용 중인 래퍼/버전/템플릿**에 따라 내부 동작(특히 tool calling 포맷)이 조금씩 다릅니다.
> - 그래도 “도구 호출 → 도구 실행 → 최종 답변 생성”이라는 큰 흐름은 동일합니다.
> 
> 아래는 **“서울 날씨 어때?” → 최종 답변**이 나오기까지의 전형적인 시나리오(내부 흐름)입니다.
> 
> ---
> 
> ## 1) 사용자 입력이 들어옴
> 사용자가 아래 메시지를 보냅니다.
> 
> - User: “서울 날씨 어때?”
> 
> 에이전트는 `invoke()`에 전달된 `messages`를 보고, system prompt도 함께 읽습니다.
> 
> - System prompt 핵심:  
  > “날씨 물어보면 get_weather tool을 사용해라. 친절하고 명확하게 답해라.”
> 
> ---
> 
> ## 2) 모델이 “도구를 써야겠다”고 결정
> 모델(LLM)은 system prompt 지시 때문에 보통 이런 판단을 합니다:
> 
> - 사용자가 “날씨”를 물었다 → 도구(get_weather)를 호출하는 게 맞다.
> - 그럼 도구에 넣을 인자를 채워야 한다.
> - 문장에서 도시/지역은 “서울”이다.
> 
> 여기서 중요한 포인트는:
> 
> - **도구 함수 시그니처가 `get_weather(location: str)`** 이기 때문에  
  > 모델은 “location이라는 문자열 인자 하나를 채우면 되네”라고 이해합니다.
> - 사용자의 문장 “서울 날씨 어때?”에서 **location 슬롯에 들어갈 값으로 “서울”을 추출**합니다.
> 
> ---
> 
> ## 3) 모델이 도구 호출(tool call)을 구성
> 이 단계에서 모델은 “최종 자연어 답변”을 바로 쓰지 않고, 내부적으로 다음과 같은 형태의 “도구 호출 요청”을 만듭니다(형식은 프레임워크/버전에 따라 다르지만 개념은 동일).
> 
> - Tool call: `get_weather`
> - Arguments: `{"location": "서울"}`
> 
> 즉, **모델이 스스로 `location="서울"`을 채워서 호출을 만든 것**이 “print에 서울이 찍힌” 직접 원인입니다.
> 
> ---
> 
> ## 4) LangChain 런타임이 실제 파이썬 함수를 실행
> 이제 에이전트 런타임(LangChain)이 모델의 도구 호출 요청을 받아서, 등록된 파이썬 함수 `get_weather`를 실행합니다.
> 
> ```python
> get_weather(location="서울")
> ```
> 
> 그래서 함수 첫 줄의:
> 
> ```python
> print(location)
> ```
> 
> 이 실행되며 콘솔에
> 
> - 서울
> 
> 이 찍힙니다.
> 
> ---
> 
> ## 5) 도구 함수가 결과를 반환
> 그 다음 함수는 `weather_data`에서 값을 찾아 반환합니다.  
> (올려주신 예제 코드에 따르면 실제로는 문자열이 아니라 dict를 반환하는 형태인데, 일단 “도구 실행 결과가 반환된다”는 흐름이 핵심입니다.)
> 
> 예: (개념적으로)
> - Tool result: `"맑음, 25°C, 습도 60%"` 또는 그에 준하는 구조화된 값
> 
> ---
> 
> ## 6) 도구 결과가 “대화 히스토리”에 다시 들어감
> LangChain은 도구 실행 결과를 **툴 결과 메시지(tool message)**로 대화에 추가합니다.
> 
> 대화 메시지 흐름을 단순화하면 대략 이렇게 됩니다:
> 
> 1. system: (너는 날씨 도우미, 날씨 질문엔 tool 써라…)
> 2. user: “서울 날씨 어때?”
> 3. assistant: (tool call: get_weather, args: location=서울)
> 4. tool: (result: 맑음, 25°C, 습도 60%)
> 5. assistant: (최종 답변 생성)
> 
> 즉, 모델은 **툴 결과를 “새로운 근거 데이터”로 받아** 마지막 응답을 작성합니다.
> 
> ---
> 
> ## 7) 모델이 최종 자연어 답변을 생성
> 이제 모델은 tool result를 보고 system prompt의 “friendly and clear” 지침에 맞게 문장을 자연스럽게 다듬어 최종 답변을 씁니다.
> 
> 예를 들면:
> 
> - “서울의 현재 날씨는 맑고 기온은 25°C이며, 습도는 60%입니다. 좋은 하루 보내세요!”
> 
> 여기서 일어나는 일은:
> 
> - 도구 결과(“맑음, 25°C, 습도 60%”)를 그대로 복붙하지 않고,
> - **문장형으로 재구성(자연어 생성)** 해서 사용자에게 전달합니다.
> 
> ---
> 
> # 요약: “서울”은 어디서 왔나?
> - 사용자의 질문에 이미 “서울”이 포함되어 있었고,
> - LLM이 그걸 읽고 **도구 인자 `location`에 매핑해서** `get_weather(location="서울")` 호출을 만들었기 때문에,
> - 함수 안에서 `print(location)` 했을 때 “서울”이 출력된 것입니다.
> 
> ---
> 
> 원하시면, 같은 흐름을 **실제 LangChain의 tool calling 메시지 포맷(어떤 필드에 어떻게 들어가는지)** 기준으로 더 “로그처럼” 재현해드릴게요.  
> 지금 사용 중인 `create_agent`가 **LangChain 버전/구현체가 정확히 무엇인지**(예: LCEL `create_tool_calling_agent` 계열인지, LangGraph인지, 혹은 다른 래퍼인지)만 알려주시면, 그 구현에 맞춰 더 정확한 시나리오로 써드릴 수 있어요.
</details>

## Agent 개요
LangChain 1.0에서는 create_agent 함수를 통해 프로덕션 환경에서 바로 사용할 수 있는 에이전트를 제공합니다. 
이 장에서는 Agent의 개념과 create_agent API의 핵심 구성 요소를 살펴봅니다

### Agent란?
Agent(에이전트)는 언어 모델(LLM)이 도구(Tools)를 활용하여 목표를 달성하는 시스템입니다. 
에이전트는 다음과 같은 특징을 갖습니다.

* 사용자의 질문을 분석하고 어떤 도구를 사용할지 스스로 결정
* 도구 실행 결과를 바탕으로 추가 작업 수행 여부 판단
* 최종 답변을 생성할 때까지 반복적으로 추론과 행동을 수행

이러한 패턴을 ReAct(Reasoning + Acting) 패턴이라고 합니다.

#### ReAct 패턴 예시

`사용자`: "최근 테슬라 주가를 검색하고 1년 수익률을 계산해줘"

`[에이전트 추론]` "주가 정보가 필요하므로 웹 검색 도구를 사용해야겠다"
`[도구 실행]` search_web("테슬라 주가 2024")
`[결과 관찰]` "현재 주가: $250, 1년 전 주가: $200"

`[에이전트 추론]` "수익률 계산이 필요하므로 계산 도구를 사용해야겠다"
`[도구 실행]` calculate("(250-200)/200 * 100")
`[결과 관찰]` "25%"

`[에이전트 추론]` "필요한 정보를 모두 얻었으니 최종 답변을 생성하겠다"
`[최종 답변]` "테슬라의 1년 수익률은 25%입니다."

In [ ]:
# create_agent 기본 사용법
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o")

# 에이전트 생성
agent = create_agent(
    model=llm,
    tools=[],  # 도구 리스트
    system_prompt="당신은 유능한 어시스턴트입니다."
)

# 에이전트 실행
result = agent.invoke({
    "messages": [{"role": "user", "content": "안녕하세요!"}]
})

print(result["messages"][-1].content)

* 동적 모델 선택
  * 런타임에 상태나 컨텍스트에 따라 모델을 동적으로 선택해야 할 때는 @wrap_model_call 미들웨어를 사용합니다.

In [ ]:
from langchain.agents.middleware import wrap_model_call
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 풀 정의
models = {
    "fast": ChatOpenAI(model="gpt-4o-mini"),
    "smart": ChatOpenAI(model="gpt-4o"),
}

@wrap_model_call
def select_model(request, state):
    """복잡도에 따라 모델 동적 선택"""
    messages = state.get("messages", [])

    # 메시지가 길거나 복잡하면 고성능 모델 사용
    if len(messages) > 10 or any("분석" in str(m) for m in messages):
        request.model = models["smart"]
    else:
        request.model = models["fast"]

    return request

agent = create_agent(
    model=models["fast"],  # 기본 모델
    tools=[search_tool],
    middleware=[select_model]
)


### Tools (도구)
에이전트가 실제 작업을 수행할 수 있게 해주는 기능입니다. 
도구를 통해 에이전트는 다음과 같은 작업을 수행할 수 있습니다.

* 웹 검색으로 실시간 정보 조회
* 코드 실행 및 계산
* 데이터베이스 쿼리
* 외부 API 호출

In [4]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults

# 내장 도구
search_tool = TavilySearchResults(max_results=3)

# 커스텀 도구
@tool
def calculate(expression: str) -> str:
    """수학 표현식을 계산합니다."""
    return str(eval(expression))

# 여러 도구를 에이전트에 등록
agent = create_agent(
    model=llm,
    tools=[search_tool, calculate],
    system_prompt="당신은 검색과 계산을 수행하는 어시스턴트입니다."
)


/var/folders/d3/h50bjcxd1nxb6d98lbh8jhgw0000gn/T/ipykernel_81535/1239173190.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=3)


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

에이전트는 도구를 다음과 같이 활용합니다.

* 여러 도구를 순차적으로 호출 (이전 결과를 다음 도구 입력으로 사용)
* 독립적인 도구를 병렬로 호출 (효율성 향상)
* 도구 실행 실패 시 재시도 또는 대체 도구 사용
* 도구 실행 결과를 상태에 저장하여 이후 참조

### System Prompt
에이전트의 역할과 동작 방식을 정의하는 지침입니다.

* 효율적인 시스템 프롬프트 작성
    역할 정의: 에이전트가 수행할 역할을 명확히 기술
    작업 방식: 도구 사용 순서와 방법을 안내
    제약 조건: 하지 말아야 할 것들을 명시
    출력 형식: 답변 형식을 지정 (마크다운, JSON 등)

In [ ]:
system_prompt = """당신은 금융 분석 전문 어시스턴트입니다.

## 역할
- 주식, 채권, 암호화폐 등 금융 정보를 분석합니다.
- 최신 시장 동향을 검색하여 정확한 정보를 제공합니다.

## 작업 방식
1. 사용자 질문을 분석합니다.
2. 필요한 정보를 웹 검색으로 조회합니다.
3. 수치 계산이 필요하면 계산 도구를 사용합니다.
4. 결과를 종합하여 명확하게 답변합니다.

## 주의사항
- 투자 조언은 제공하지 않습니다.
- 출처를 명시합니다.
"""

agent = create_agent(
    model=llm,
    tools=[search_tool, calculate],
    system_prompt=system_prompt
)

In [ ]:
# 동적 시스템 프롬프트
# 런타임에 상태나 사용자 정보에 따라 프롬프트를 동적으로 생성해야 할 때는 @dynamic_prompt 미들웨어를 사용합니다
from langchain.agents.middleware import dynamic_prompt
from langchain.agents import create_agent

@dynamic_prompt
def personalized_prompt(request, state):
    """사용자 정보에 따라 프롬프트 동적 생성"""
    user_name = state.get("user_name", "사용자")
    language = state.get("language", "ko")

    if language == "ko":
        return f"""안녕하세요 {user_name}님, 저는 AI 어시스턴트입니다.

## 역할
- 사용자의 질문에 친절하게 답변합니다.
- 필요시 도구를 사용하여 정보를 조회합니다.

## 주의사항
- 항상 한국어로 답변합니다.
- 존댓말을 사용합니다."""
    else:
        return f"Hello {user_name}, I'm your AI assistant."

agent = create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[personalized_prompt]  # 동적 프롬프트 미들웨어
)

- 대화 히스토리 관리
여러 턴의 대화를 처리하려면 메시지 히스토리를 유지합니다.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# 대화 히스토리 초기화
messages = []

# 첫 번째 대화
messages.append(HumanMessage(content="서울 날씨 알려줘"))
result = agent.invoke({"messages": messages})

# 응답을 히스토리에 추가
messages.extend(result["messages"])

# 후속 질문 (컨텍스트 유지)
messages.append(HumanMessage(content="그럼 우산 필요할까?"))
result = agent.invoke({"messages": messages})

print(result["messages"][-1].content)


## 내장 도구
도구(Tools)는 모델이 호출하도록 설계된 유틸리티입니다. 
도구의 입력은 모델이 생성하도록 설계되었고, 출력은 모델에 다시 전달되도록 설계되었습니다. 
툴킷(Toolkit)은 함께 사용되도록 의도된 도구들의 모음입00ㅔ니다.

LangChain은 다양한 작업을 수행하기 위한 여러 내장 도구를 제공합니다. 
이러한 도구들은 언어 모델의 기능을 확장하고 외부 데이터 소스와의 상호작용을 가능하게 합니다.
https://wikidocs.net/261567

## 커스텀 도구
LangChain v1.0에서는 @tool 데코레이터를 사용하여 커스텀 도구를 쉽게 만들고, create_agent()를 통해 에이전트와 통합할 수 있습니다
https://wikidocs.net/261571

### 에러 처리미들 웨어

In [ ]:
# 기본 에러 처리 미들웨어
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage
import requests

@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 중 발생하는 에러를 처리하는 미들웨어"""
    try:
        # 원래 도구 실행
        return handler(request)
    except requests.exceptions.HTTPError as e:
        # HTTP 에러 처리
        return ToolMessage(
            content=f"날씨 API 요청 실패: {e.response.status_code}",
            tool_call_id=request.tool_call["id"]
        )
    except requests.exceptions.RequestException as e:
        # 네트워크 에러 처리
        return ToolMessage(
            content=f"네트워크 오류 발생: {str(e)}",
            tool_call_id=request.tool_call["id"]
        )
    except Exception as e:
        # 기타 에러 처리
        return ToolMessage(
            content=f"도구 실행 오류: {str(e)}",
            tool_call_id=request.tool_call["id"]
        )

# 미들웨어가 적용된 에이전트 생성
agent = create_agent(
    model=llm,
    tools=[get_weather],
    middleware=[handle_tool_errors],
    system_prompt="당신은 날씨 정보 제공 전문 어시스턴트입니다."
)


In [ ]:
# 재시도 미들웨어
# 실패 시 자동으로 재시도하는 미들웨어 추가
import time
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

@wrap_tool_call
def retry_on_failure(request, handler, max_retries=3, delay=1):
    """실패 시 재시도하는 미들웨어"""
    for attempt in range(max_retries):
        try:
            return handler(request)
        except Exception as e:
            if attempt == max_retries - 1:
                # 마지막 시도 실패 시 에러 메시지 반환
                return ToolMessage(
                    content=f"{max_retries}번 재시도 후 실패: {str(e)}",
                    tool_call_id=request.tool_call["id"]
                )
            # 재시도 전 대기
            time.sleep(delay * (attempt + 1))

# 여러 미들웨어 적용
agent = create_agent(
    model=llm,
    tools=[get_weather],
    middleware=[retry_on_failure, handle_tool_errors],
    system_prompt="당신은 날씨 정보 제공 전문 어시스턴트입니다."
)


미들웨어는 리스트 순서대로 실행됩니다. 
위 예제에서는
1. retry_on_failure가 먼저 실행되어 재시도 로직 처리 
2. handle_tool_errors가 두 번째로 실행되어 세부 에러 처리

따라서 재시도가 먼저, 에러 처리가 나중에 오는 것이 일반적입니다.

## ToolRuntime & 컨택스트
ToolRuntime은 도구 함수에 자동으로 주입되는 특별한 파라미터입니다. 
LLM에게는 노출되지 않으면서 도구 내부에서 다음 정보에 접근할 수 있게 해줍니다.

|속성|설명|
|--|--|
|state|에이전트 상태 (대화 히스토리, 커스텀 필드 등)|
|context|불변 컨텍스트 (사용자 ID, 세션 정보 등)|
|store|장기 메모리 저장소|
|stream_writer|실시간 스트리밍 출력|
|config|RunnableConfig 설정|
|tool_call_id|현재 도구 호출 ID|

In [ ]:
# 기본 사용법
# 도구 함수에 runtime: ToolRuntime 파라미터를 추가하면 자동으로 주입됩니다.
from langchain.tools import tool, ToolRuntime

@tool
def get_user_preference(category: str, runtime: ToolRuntime) -> str:
    """사용자의 선호도를 조회합니다.

    Args:
        category: 조회할 카테고리 (food, music, etc.)
    """
    # runtime에서 사용자 ID 가져오기
    user_id = runtime.context.get("user_id", "anonymous")

    # store에서 사용자 선호도 조회
    preferences = runtime.store.get(user_id, {})

    return f"{user_id}님의 {category} 선호도: {preferences.get(category, '정보 없음')}"

# runtime: ToolRuntime은 LLM에게 노출되지 않습니다
# 함수 시그니처에 있어도 LLM은 이 파라미터를 생성하지 않습니다
# 시스템이 자동으로 런타임 정보를 주입합니다
# 기존의 InjectedState, InjectedStore, InjectedToolCallId를 대체


In [9]:
# State (상태) 접근
# 상태 읽기
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from typing import TypedDict, Annotated
from langgraph.graph import add_messages

# 커스텀 상태 정의
class MyAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    user_name: str
    visit_count: int

@tool
def greet_user(runtime: ToolRuntime) -> str:
    """사용자에게 맞춤 인사를 합니다."""
    state = runtime.state

    user_name = state.get("user_name", "손님")
    visit_count = state.get("visit_count", 0)

    if visit_count == 0:
        return f"안녕하세요, {user_name}님! 처음 오셨군요. 환영합니다!"
    else:
        return f"반갑습니다, {user_name}님! {visit_count}번째 방문이시네요."

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# 커스텀 상태를 사용하는 에이전트 생성
agent = create_agent(
    model=llm,
    tools=[greet_user],
    state_schema=MyAgentState,
    system_prompt="사용자 맞춤 서비스를 제공하는 어시스턴트입니다."
)


In [ ]:
# 상태 업데이트
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

@tool
def increment_counter(runtime: ToolRuntime) -> Command:
    """방문 카운터를 증가시킵니다."""
    current_count = runtime.state.get("visit_count", 0)

    # 상태 업데이트를 포함한 Command 반환
    return Command(
        update={
            "visit_count": current_count + 1
        }
    )

@tool
def save_preference(category: str, value: str, runtime: ToolRuntime) -> Command:
    """사용자 선호도를 저장합니다.

    Args:
        category: 카테고리 (food, music, movie 등)
        value: 선호 값
    """
    current_prefs = runtime.state.get("preferences", {})
    current_prefs[category] = value

    return Command(
        update={"preferences": current_prefs},
        # 선택적: 다음 단계로 이동
        # goto="next_node"
    )

### Context (컨텍스트) 접근
컨텍스트는 에이전트 실행 시 전달되는 불변(immutable) 정보입니다. 
사용자 ID, 세션 정보, API 키 등을 안전하게 전달할 수 있습니다.

LangChain 1.0에서는 context 파라미터를 직접 전달하는 새로운 방식을 제공합니다. 
기존 config["configurable"] 방식보다 타입 안전하고 명확합니다.

In [ ]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent

# Context 타입 정의 (dataclass 권장)
@dataclass
class AppContext:
    user_id: str
    session_id: str
    api_key: str
    region: str = "KR"

@tool
def get_user_info(runtime: ToolRuntime[AppContext]) -> str:
    """사용자 정보를 조회합니다."""
    # 타입 안전한 컨텍스트 접근
    ctx = runtime.context
    return f"User: {ctx.user_id}, Region: {ctx.region}"

# 에이전트 생성 시 context_schema 지정
agent = create_agent(
    model=llm,
    tools=[get_user_info],
    context_schema=AppContext,  # 컨텍스트 스키마 지정
    system_prompt="사용자 맞춤 서비스 어시스턴트입니다."
)

# 실행 시 context 파라미터로 직접 전달
result = agent.invoke(
    {"messages": [{"role": "user", "content": "내 정보 알려줘"}]},
    context=AppContext(
        user_id="user_123",
        session_id="sess_abc",
        api_key="sk-xxx"
    )
)

print(result)
result["messages"][-1].content

{'messages': [HumanMessage(content='내 정보 알려줘', additional_kwargs={}, response_metadata={}, id='d1b9a550-62f7-49f9-9385-88b31bb8d1e4'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 53, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-D2ufXiXGm57rWw7o1x7YU3fnsFmHt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c0398-aa2c-7ac0-b270-7b2c6cace263-0', tool_calls=[{'name': 'get_user_info', 'args': {}, 'id': 'call_rzRvn7d5RXADCSDbjbVy09vH', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 53, 'output_tokens': 11, 'total_tokens': 64, 'input_

'당신의 사용자 정보는 다음과 같습니다:\n- 사용자 ID: user_123\n- 지역: 대한민국(KR)'

In [13]:
# 인증 정보 전달
import httpx
from langchain.tools import tool, ToolRuntime

@tool
async def fetch_user_data(endpoint: str, runtime: ToolRuntime) -> str:
    """사용자 데이터를 외부 API에서 조회합니다.

    Args:
        endpoint: API 엔드포인트 경로
    """
    # 컨텍스트에서 인증 정보 가져오기
    api_key = runtime.context.get("api_key")
    base_url = runtime.context.get("api_base_url", "https://api.example.com")

    if not api_key:
        return "API 키가 설정되지 않았습니다."

    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{base_url}/{endpoint}",
            headers={"Authorization": f"Bearer {api_key}"}
        )
        return response.json()


In [32]:
# Store (장기메모리) 접근
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore

@tool
def remember_fact(key: str, value: str, runtime: ToolRuntime) -> str:
    """사실을 장기 메모리에 저장합니다.

    Args:
        key: 저장할 키
        value: 저장할 값
    """
    print(1)
    user_id = runtime.context.get("user_id", "default")

    # Store에 저장
    namespace = ("user_facts", user_id)
    runtime.store.put(namespace, key, {"value": value})

    return f"'{key}: {value}'를 기억했습니다."

@tool
def recall_fact(key: str, runtime: ToolRuntime) -> str:
    """장기 메모리에서 사실을 조회합니다.
    
    Args:
        key: 조회할 키
    """
    print(2)
    user_id = runtime.context.get("user_id", "default")
    # Store에서 조회
    namespace = ("user_facts", user_id)
    item = runtime.store.get(namespace, key)

    if item:
        return f"'{key}'에 대한 기억: {item.value['value']}"
    else:
        return f"'{key}'에 대한 기억이 없습니다."

# Store 설정
store = InMemoryStore()

# Store가 연결된 에이전트 생성
agent = create_agent(
    model=llm,
    tools=[remember_fact, recall_fact],
    store=store,
    system_prompt="사용자의 정보를 기억하는 어시스턴트입니다."
)

# result = agent.invoke({
#     "messages": [{"content": "사용자 이름은 누구?", "role": "user"}],
# },
#     context= {
#         "user_id": "서혁택"
#     })
result = agent.invoke({
    "messages": [{"content": "나는 양지웅이야 나를 기억하겠어?", "role": "user"}],
},
    context= {
        "user_id": "서혁택"
    })

result["messages"][-1].content



/Users/seohyuktaek/Documents/study/langchain/venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value={'user_id': '서혁택'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(


2


/Users/seohyuktaek/Documents/study/langchain/venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value={'user_id': '서혁택'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(


1


'안녕하세요, 양지웅님! 이제 당신의 이름을 기억했습니다.'

In [ ]:
# Store 활용 패턴
from langchain.tools import tool, ToolRuntime

@tool
def update_user_profile(field: str, value: str, runtime: ToolRuntime) -> str:
    """사용자 프로필을 업데이트합니다.

    Args:
        field: 업데이트할 필드 (name, email, preference 등)
        value: 새 값
    """
    user_id = runtime.context.get("user_id")
    namespace = ("profiles", user_id)

    # 기존 프로필 조회
    existing = runtime.store.get(namespace, "profile")
    profile = existing.value if existing else {}

    # 필드 업데이트
    profile[field] = value

    # 저장
    runtime.store.put(namespace, "profile", profile)

    return f"프로필 업데이트 완료: {field} = {value}"

@tool
def get_user_profile(runtime: ToolRuntime) -> str:
    """사용자 프로필 전체를 조회합니다."""
    user_id = runtime.context.get("user_id")
    namespace = ("profiles", user_id)

    existing = runtime.store.get(namespace, "profile")

    if existing:
        profile = existing.value
        lines = [f"- {k}: {v}" for k, v in profile.items()]
        return "사용자 프로필:\n" + "\n".join(lines)
    else:
        return "저장된 프로필이 없습니다."


In [33]:
# Stream Writer ( 실시간 스트리밍)
# stream_writer를 사용하면 도구 실행 중에 실시간으로 진행 상황을 스트리밍할 수 있습니다.
import time
from langchain.tools import tool, ToolRuntime

@tool
def long_running_task(steps: int, runtime: ToolRuntime) -> str:
    """여러 단계의 작업을 수행합니다.

    Args:
        steps: 수행할 단계 수
    """
    writer = runtime.stream_writer

    for i in range(1, steps + 1):
        # 진행 상황 스트리밍
        writer({"progress": f"단계 {i}/{steps} 처리 중..."})
        time.sleep(0.5)  # 작업 시뮬레이션

    writer({"status": "완료"})
    return f"총 {steps}단계 작업이 완료되었습니다."

@tool
def fetch_multiple_sources(sources: list[str], runtime: ToolRuntime) -> str:
    """여러 소스에서 데이터를 수집합니다.

    Args:
        sources: 조회할 소스 목록
    """
    writer = runtime.stream_writer
    results = []

    for i, source in enumerate(sources, 1):
        # 현재 진행 상황 스트리밍
        writer({
            "type": "progress",
            "message": f"[{i}/{len(sources)}] {source} 조회 중..."
        })

        # 실제 데이터 수집 로직
        data = f"{source}에서 수집한 데이터"
        results.append(data)

        # 개별 결과 스트리밍
        writer({
            "type": "partial_result",
            "source": source,
            "data": data
        })

    return "\n".join(results)


In [35]:
agent = create_agent(
    model=llm,
    tools=[long_running_task, fetch_multiple_sources],
    system_prompt="데이터 수집 어시스턴트입니다."
)

# stream_mode="custom"으로 실행해야 stream_writer 출력을 받을 수 있음
for event in agent.stream(
    {"messages": [{"role": "user", "content": "5단계 작업 실행해줘"}]},
    stream_mode="custom"
):
    print(event)


{'progress': '단계 1/5 처리 중...'}
{'progress': '단계 2/5 처리 중...'}
{'progress': '단계 3/5 처리 중...'}
{'progress': '단계 4/5 처리 중...'}
{'progress': '단계 5/5 처리 중...'}
{'status': '완료'}


In [37]:
# ToolCallID
# tool_call_id는 현재 도구 호출의 고유 식별자입니다. 디버깅이나 로깅에 유용합니다.
import logging
from langchain.tools import tool, ToolRuntime

logger = logging.getLogger(__name__)

@tool
def tracked_operation(action: str, runtime: ToolRuntime) -> str:
    """추적 가능한 작업을 수행합니다.

    Args:
        action: 수행할 작업
    """
    tool_call_id = runtime.tool_call_id

    # 로깅에 tool_call_id 포함
    logger.info(f"[{tool_call_id}] 작업 시작: {action}")

    # 작업 수행
    result = f"'{action}' 작업 완료"

    logger.info(f"[{tool_call_id}] 작업 완료")

    return result

agent = create_agent(
    model=llm,
    tools=[tracked_operation],
    system_prompt="추적 가능한 작업을 수행하는 어시스턴트입니다."
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "추적 가능한 작업 실행해줘"}]
})

result["messages"][-1].content



'어떤 작업을 추적할지를 명시해주시면, 해당 작업을 실행하여 추적할 수 있습니다. 수행할 작업의 이름이나 설명을 알려주세요.'